In [15]:
import pandas as pd
import numpy as np
import os
import healpy as hp
from astroquery.vizier import Vizier
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time
import math


In [16]:
def RenameColumns(table, catalogue):
    global info
    
    old_columns = info.drop(['link', 'pecularity'], axis = 1).loc[catalogue]
    new_columns = info.drop(['link', 'pecularity'], axis = 1).columns
        
    old_columns = old_columns.tolist()
    new_columns = new_columns.tolist()
    
        
    for i in range(len(new_columns)):
        table = table.rename(columns={old_columns[i]: new_columns[i]})
    return table

In [17]:
def errors_2MASS(row):
    a = row['errMaj']
    b = row['errMin']
    angle = row['errPA']
    row['errMaj'] = abs(a * b) / (math.sqrt((a ** 2) * (math.cos(math.radians(angle)) ** 2) + 
                                            (b ** 2) * (math.sin(math.radians(angle)) ** 2)))
    row['errMin'] = abs(a * b) / (math.sqrt((a ** 2) * (math.sin(math.radians(angle)) ** 2) + 
                                            (b ** 2) * (math.cos(math.radians(angle)) ** 2)))
    return row                       

In [18]:
def epoch_UCAC4(row):
    a = row['EpRA']
    b = row['EpDE']
    row['EpRA'] = (a + b)/2
    return row

In [19]:
def CatDataFrame(star, catalogue):
    global info
    global radsec
    try:
        link = info.loc[catalogue]['link']
        viz = Vizier(columns=['all'])
        table = viz.query_region(coord.SkyCoord(ra=star.ra, dec=star.dec, unit=(u.deg, u.deg), frame='icrs'),
                            radius=radsec*u.arcsec, catalog=link)[0]
        
        df = pd.DataFrame.from_records(table.as_array())
        
        pecularities = info.pecularity[catalogue]
        pecs = pecularities.split('|')
            
        if (not pecs[0] == 'mas') & ('ra_error' in info.columns) & ('dec_error' in info.columns):
                
            column1 = info ['ra_error'][catalogue]
            column2 = info ['dec_error'][catalogue]
                    
            df[column1] = df[column1].apply(lambda x: x*1000)
            df[column2] = df[column2].apply(lambda x: x*1000)
                    
            if len(pecs[0].split(':')) == 2:
                df[['errMaj', 'errMin', 'errPA']] = df[['errMaj', 'errMin', 'errPA']].apply(errors_2MASS, axis = 1)
                        
                
        if (not pecs[1] == 'jy') & ('ref_epoch' in info.columns):
            ps = pecs[1].split(':')
                    
            if len(ps) == 2:
                column = info ['ref_epoch'][catalogue]
                df[column] = df[column].apply(lambda x: Time((x+ int(ps[0])), format=ps[1]).jyear)
            else:
                df[['EpRA','EpDE']] = df[['EpRA','EpDE']].apply(epoch_UCAC4, axis = 1)

        cols = info.loc[catalogue].tolist()[:-2]
        df = df[['_r'] + cols]
        df = RenameColumns(df, catalogue)
        df['delta_mag'] = df.mag
        df.delta_mag = df.delta_mag.apply(lambda x: abs(x-star.mag))
        first_col = df.pop('delta_mag')
        df.insert(0, 'delta_mag', first_col)
        df.delta_mag = round(df.delta_mag, 2)
        df._r = round(df._r, 4)
        df.delta_mag = round(df.delta_mag, 2)
        df.ref_epoch = round(df.ref_epoch, 1)
        
        #print(catalogue)# + ': success')
        return [df, True]
    
    except Exception as e:
        #print(catalogue + '\t' + ' ERROR:', e)
        return ['NaN', False]

In [20]:
def best_match(df):
    global radsec
    global start_delta_mag
    number = 0
    if len(df) > 1:
        no_mag = True
        minimal = start_delta_mag
        for i in range(len(df)):
            s = df.iloc[i]
            if s.delta_mag < minimal:
                no_mag = False
                minimal = s.delta_mag
                number = i
        no_mag = True# uncomment if _r
        if no_mag:
            distance = radsec
            for i in range(len(df)):
                s = df.iloc[i]
                if (s._r < distance)&(s.delta_mag < (start_delta_mag+minimal)/2):
                    distance = s._r
                    number = i
    s = df.drop('mag', axis = 1).iloc[number].apply(str)
    #display(s)
    list_s = s.tolist()
    #print('|'.join(list_s))
    return '|'.join(list_s)

In [21]:
'''Global variables.'''
radsec = 30
start_delta_mag = 10
info = pd.read_csv('cat_info.csv').fillna('NaN')
gaia_columns = info.columns.tolist()[:-2]+ ['pmra', 'pmdec']
info = info.drop(['ra_error', 'dec_error'], axis = 1)
catlist = info.index.tolist()

info = info.rename(columns={'phot_g_mean_mag': 'mag'})
display(info)

gaiapath = os.path.join('source','GaiaSource-my_close_stars.csv')
gaia_df = pd.read_csv(gaiapath)[gaia_columns]
gaia_df = gaia_df.rename(columns={'phot_g_mean_mag': 'mag'})
display(gaia_df.sample())  

,ra,dec,ref_epoch,mag,link,pecularity
CMC15,RA_ICRS,DE_ICRS,MJD-51263,r_mag,I/327/cmc15,arcsec|51263:mjd
Pan-STARRS DR1,RAJ2000,DEJ2000,_tab1_10,gmag,II/349/ps1,arcsec|0:mjd
2MASS All-Sky,RAJ2000,DEJ2000,_tab1_36,Jmag,II/246/out,arcsec:errPA|0:jd
SDSS DR12,RA_ICRS,DE_ICRS,ObsDate,gmag,V/147/sdss12,arcsec|jy
URAT1,RAJ2000,DEJ2000,Epoch,gmag,I/329/urat1,mas|jy
UCAC4,RAJ2000,DEJ2000,EpRA,gmag,I/322A/out,mas|EpDE


,ra,dec,ra_error,dec_error,ref_epoch,mag,pmra,pmdec
17619,349.912638,-64.982934,0.046471,0.047606,2015.5,14.562012,380.491836,-582.509437


In [22]:
catlist_description = []
    
for catalogue in catlist:
    catlist_decsribtion.append(catalogue+'[dm|r|ra|dec|t]')

outfile = 'catalog_positions.csv'
cat_pos_df = pd.DataFrame(np.empty([0, 10]), columns=['ra', 'dec', 'pmra', 'pmdec']+catlist_description)
cat_pos_df.to_csv(outfile, index=False)

print('from '+str(len(gaia_df))+' stars completed:')
print()

for i in range(len(gaia_df)):
    row = []
    #if i > 3:
    #    break
    print(i+1, end=', ')
    star = gaia_df.iloc[i]
    row.append(star.ra)
    row.append(star.dec)
    row.append(star.pmra)
    row.append(star.pmdec)
    
    for catalogue in catlist:
        df, check = CatDataFrame(star, catalogue)
        #continue
        if check:
            row.append(best_match(df))
        else:
            row.append('')
    row = pd.Series(row, index = ['ra', 'dec', 'pmra', 'pmdec']+catlist_description).to_frame()
    row = row.T
    cat_pos_df = pd.read_csv(outfile)
    cat_pos_df = cat_pos_df.append(row, ignore_index=True)
    cat_pos_df.to_csv(outfile, index=False)
#cat_pos_df

ValueError: Empty data passed with indices specified.

In [ ]:
cat_pos_df[cat_pos_df.index == 0]

In [1]:
#'ra','dec','pmra','pmdec','CMC15[dm|r|ra|dec|t]','Pan-STARRS DR1[dm|r|ra|dec|t]','2MASS All-Sky[dm|r|ra|dec|t]','SDSS DR12[dm|r|ra|dec|t]','URAT1[dm|r|ra|dec|t]','UCAC4[dm|r|ra|dec|t]'